# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# pip install gmaps

In [60]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [61]:
file = "cities_pd.csv"

cities_pd = pd.read_csv(file)

cities_pd = cities_pd[['city', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
                      'Wind Speed', 'Country', 'Date']]

cities_pd

,city,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,codrington,-38.2667,141.9667,285.87,55,8,1.50,AU,1659932146
1,rikitea,-23.1203,-134.9692,294.44,77,100,16.51,PF,1659932146
2,port alfred,-33.5906,26.8910,287.14,99,88,0.45,ZA,1659932112
3,union de san antonio,21.1000,-101.9667,289.85,68,91,1.65,MX,1659932148
4,butaritari,3.0707,172.7902,300.76,75,92,5.43,KI,1659932148
...,...,...,...,...,...,...,...,...,...
550,manggar,-2.8833,108.2667,302.43,67,4,5.85,ID,1659933135
551,kolobovo,56.7020,41.3417,291.64,90,99,2.62,RU,1659933136
552,kochki,54.3358,80.4806,293.34,64,57,3.73,RU,1659933137
553,gangapur,26.4833,76.7167,304.33,59,75,2.99,IN,1659933138


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [62]:
gmaps.configure(api_key=g_key)

locations = cities_pd[['Lat' , 'Lng']]

humidity = cities_pd['Humidity'].astype(float)

print(len(locations), len(humidity))

555 555


In [67]:
fig = gmaps.figure(zoom_level=2, center=(0, 0))

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, max_intensity=50, dissipating=False, point_radius=1)

fig.add_layer(heat_layer)

plt.fig

AttributeError: module 'matplotlib.pyplot' has no attribute 'fig'

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [55]:
cities_weather_complete = pd.DataFrame(cities_pd.dropna(how='any'))

# cities_weather_complete

len(cities_weather_complete)
# len(cities_pd)

553

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [36]:
import time

hotel_df = pd.DataFrame(cities_weather_complete[['city', 'Country', 'Lat', 'Lng']])
hotel_df['Hotel Name'] =""
params = {"key": g_key,
          "rankedby": "distance",
          "type": "hotel",
          "keyword": "hotel",
         "radius": "5000"}
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    city = row['city']
    params['location'] = f"{lat} {lng}"
    
    response = requests.get(base_url, params=params).json()

    results = response['results']
    
    try:
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    except  (KeyError, IndexError):
        hotel_df.loc[index, 'Hotel Name'] = "No hotel found"
#     time.sleep(1)
    print(f"Loading record index {index}")
print("----------------------------")
print("------- TASK COMPLETED -----")
print("----------------------------")

Loading record index 0
Loading record index 1
Loading record index 2
Loading record index 3
Loading record index 4
Loading record index 5
Loading record index 6
Loading record index 7
Loading record index 8
Loading record index 9
Loading record index 10
Loading record index 11
Loading record index 12
Loading record index 13
Loading record index 14
Loading record index 15
Loading record index 16
Loading record index 17
Loading record index 18
Loading record index 19
Loading record index 20
Loading record index 21
Loading record index 22
Loading record index 23
Loading record index 24
Loading record index 25
Loading record index 26
Loading record index 27
Loading record index 28
Loading record index 29
Loading record index 30
Loading record index 31
Loading record index 32
Loading record index 33
Loading record index 34
Loading record index 35
Loading record index 36
Loading record index 37
Loading record index 38
Loading record index 39
Loading record index 40
Loading record index 41
Lo

Loading record index 334
Loading record index 335
Loading record index 336
Loading record index 337
Loading record index 338
Loading record index 339
Loading record index 340
Loading record index 341
Loading record index 342
Loading record index 343
Loading record index 344
Loading record index 345
Loading record index 346
Loading record index 347
Loading record index 348
Loading record index 349
Loading record index 350
Loading record index 351
Loading record index 352
Loading record index 353
Loading record index 354
Loading record index 355
Loading record index 356
Loading record index 357
Loading record index 358
Loading record index 359
Loading record index 360
Loading record index 361
Loading record index 362
Loading record index 363
Loading record index 364
Loading record index 365
Loading record index 366
Loading record index 367
Loading record index 368
Loading record index 369
Loading record index 370
Loading record index 371
Loading record index 372
Loading record index 373


In [56]:
hotel_df['Hotel Name']


0      Codrington Gardens Bed And Breakfast
1                   Pension Bianca & Benoit
2                    Royal St Andrews Hotel
3                      HOTEL M. DE LA PERLA
4                            No hotel found
                       ...                 
550                     OYO 515 Oasis Hotel
551                          No hotel found
552                      Гостиница «Малина»
553         Capital O 45463 Hotel The Pearl
554                  Hotel Tropeiro da Lapa
Name: Hotel Name, Length: 553, dtype: object

In [57]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [58]:
# Hotel marker 

melbourne_coordinates = (-37.8136, 144.9631)
fig = gmaps.figure(center=melbourne_coordinates, zoom_level=5)

hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [59]:
# Add marker layer ontop of heat map
fig = gmaps.figure(center=melbourne_coordinates, zoom_level=5, map_type='HYBRID')

fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

fig

Figure(layout=FigureLayout(height='420px'))